In [3]:
!pip install selenium lxml tabulate
!apt-get update
!apt-get install -y chromium-chromedriver

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB 991.0 kB/s eta 0:00:10
   ---------------------------------------- 0.1/9.7 MB 1.1 MB/s eta 0:00:09
    --------------------------------------- 0.2/9.7 MB 1.5 MB/s eta 0:00:07
   - -------------------------------------- 0.4/9.7 MB 2.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.6/9.7 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/9.7 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.2/9.7 MB 4.0 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.7 MB 4.3 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.7 MB 4.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.7 MB 4.3 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.7 MB 4.1 MB/s eta 0:00:02
   ----------- -----


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [24]:
import os
import string
from IPython.display import Image, display
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup
from io import StringIO

def search_term_and_extract_data(TEXT_TO_SEARCH):
    # Create a new directory for the user data
    user_data_dir = "/tmp/chrome_user_data"
    os.makedirs(user_data_dir, exist_ok=True)

    # Set up Chrome options for Google Colab
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'--user-data-dir={user_data_dir}')
    options.add_argument('--profile-directory=Profile 3')

    # Initialize the WebDriver
    driver = webdriver.Chrome(options=options)

    try:
        # Step 1: Open the settings page and activate the Spanish checkbox
        driver.get("https://unterm.un.org/unterm2/settings?displayIn=es&searchin=ar&searchin=en&searchin=es&searchin=fr&searchin=ru&searchin=zh")
        wait = WebDriverWait(driver, 1)  # Increase wait time to 5 seconds

        # Print the raw HTML of the settings page
        page_source = driver.page_source
        #print(page_source)

        # Check if the Spanish checkbox is present in the HTML
        if "title=\"Spanish\" type=\"checkbox\" name=\"displayIn\"" not in page_source:
            print("Spanish checkbox not found in the HTML.")

        # Retry mechanism to ensure the checkbox is clickable
        retries = 0
        for attempt in range(retries):
            try:
                spanish_checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@title='Spanish' and @name='displayIn']")))
                spanish_checkbox.click()
                print("Spanish checkbox clicked.")
                break
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {e}")
                time.sleep(2)  # Wait before retrying
            else:
                print("Exception(\"Failed to click the Spanish checkbox after multiple attempts.\")")

        time.sleep(1)  # Wait for the checkbox to be activated

        # Click the "Update Default Settings" button
        try:
            update_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@type='submit' and contains(text(), 'Update Default Settings')]")))
            update_button.click()
            print("Update Default Settings button clicked.")
        except Exception as e:
            print(f"Failed to click the Update Default Settings button: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Continue even if it fails
        pass

    try:
        # Step 1: Go to the search URL with the specified parameters
        search_url = f"https://unterm.un.org/unterm2/en/search?searchTerm={TEXT_TO_SEARCH}&searchType=0&searchLanguages=ar&searchLanguages=en&searchLanguages=es&searchLanguages=fr&searchLanguages=ru&searchLanguages=zh&languagesDisplay=en&languagesDisplay=es&acronymSearch=true&localDBSearch=true&termTitleSearch=true&phraseologySearch=false&footnoteSearch=false&fullTextSearch=false&facetedSearch=false&buildSubjectList=true"
        driver.get(search_url)
        wait = WebDriverWait(driver, 10)  # Increase wait time to 10 seconds

        # Step 2: Click on Advanced Settings
        try:
            advanced_search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.text-dark.text-nowrap.btn-link.collapsed")))
            advanced_search_button.click()
        except Exception as e:
            print(f"CSS Failed to search-click the Advanced search button: {e}")
            driver.save_screenshot("/content/screenshot_ERROR_filters.png")

        time.sleep(2)  # Wait for the advanced search options to be visible

        # Step 3: Activate the Display in: "Spanish" button and other languages
        languages = ["Spanish", "Russian", "Chinese", "Arabic"]
        for lang in languages:
            try:
                lang_button = wait.until(EC.element_to_be_clickable((By.XPATH, f"//button[@id='ds-{lang}' and @aria-pressed='false']")))
                lang_button.click()
                print(f"Display in {lang} button clicked.")
            except Exception as e:
                print(f"Failed to click the Display in {lang} button: {e}")

        driver.save_screenshot("/content/screenshot_filters.png")

        # Step 4: Wait for the table to be visible and scrape its HTML content
        try:
            table = wait.until(EC.visibility_of_element_located((By.XPATH, "//table")))
            table_html = table.get_attribute('outerHTML')
            print("Table HTML content retrieved.")
        except Exception as e:
            print(f"Failed to retrieve the table HTML content: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the WebDriver
        driver.quit()

    # Step 5: Convert the table HTML to a dictionary
    try:
        soup = BeautifulSoup(table_html, 'html.parser')
        rows = soup.find_all('tr')
        data = []
        for row in rows:
            cols = row.find_all('td')
            if len(cols) > 0:
                row_data = {}
                for col, lang in zip(cols[1:], ["English", "French", "Spanish", "Russian", "Chinese", "Arabic"]):
                    terms = []
                    ul = col.find('ul', class_='search-result')
                    if ul:
                        for li in ul.find_all('li'):
                            span = li.find('span', class_=True, lang=True)
                            if span:
                                term = span.get_text(strip=False)
                                term_class = span['class'][0]
                                terms.append({"term": term, "termClass": term_class})
                    row_data[lang] = terms

                # Extract source information
                source_div = cols[-1].find('div', class_='record-info')
                if source_div:
                    source = source_div.find('h5').get_text(strip=False)
                    tags = [li.get_text(strip=False) for li in source_div.find_all('li')]
                    row_data["UNTerm_Source"] = {"source": source, "tags": tags}
                else:
                    row_data["UNTerm_Source"] = {"source": "", "tags": []}

                data.append(row_data)
        table_html_io = StringIO(table_html)
        df = pd.read_html(table_html_io)[0]
        markdown_table = df.to_markdown(index=False)
        #print(markdown_table)
        data_df = pd.DataFrame(data)

        return data, markdown_table, table_html, df, data_df
    except Exception as e:
        print(f"Failed to convert the table HTML to a dictionary: {e}")
        return []


In [29]:
# Example usage
TEXT_TO_SEARCH = "Türkiye"
data, markdown_table, table_html, df, data_df = search_term_and_extract_data(TEXT_TO_SEARCH)
print("\n\nDATA")
print(data)
print("\n\nMARKDOWN TABLE")
print(markdown_table)
print("\n\nTABLE HTML")
print(table_html)
print("\n\nDATAFRAME")
df

Update Default Settings button clicked.
Display in Spanish button clicked.
Display in Russian button clicked.
Display in Chinese button clicked.
Display in Arabic button clicked.
Table HTML content retrieved.


DATA
[{'English': [{'term': 'Türkiye', 'termClass': 'preferred'}, {'term': 'the Republic of Türkiye', 'termClass': 'preferred'}], 'French': [{'term': 'Türkiye (la)', 'termClass': 'preferred'}, {'term': 'la République de Türkiye', 'termClass': 'preferred'}], 'Spanish': [{'term': 'Türkiye', 'termClass': 'preferred'}, {'term': 'la República de Türkiye', 'termClass': 'preferred'}], 'Russian': [{'term': 'Турецкая Республика', 'termClass': 'preferred'}, {'term': 'Турция', 'termClass': 'preferred'}], 'Chinese': [{'term': '土耳其共和国', 'termClass': 'preferred'}, {'term': '土耳其', 'termClass': 'preferred'}], 'Arabic': [{'term': 'الجمهورية التركية', 'termClass': 'preferred'}, {'term': 'تركيا', 'termClass': 'preferred'}], 'UNTerm_Source': {'source': 'UNHQ', 'tags': ['Country names']}}, {'English

,Unnamed: 0,English (Click to sort Ascending),French (Click to sort Ascending),Spanish (Click to sort Ascending),Russian (Click to sort Ascending),Chinese (Click to sort Ascending),Arabic (Click to sort Ascending),Unnamed: 7
0,Open Details,Türkiye the Republic of Türkiye,Türkiye (la) la République de Türkiye,Türkiye la República de Türkiye,Турецкая Республика Турция,土耳其共和国 土耳其,الجمهورية التركية تركيا,UNHQ Country names
1,Open Details,Grand National Assembly of Türkiye,Grande Assemblée nationale de Turquie Grande A...,Gran Asamblea Nacional de Turquía Gran Asamble...,Великое национальное собрание Турции,土耳其大国民议会,الجمعية الوطنية الكبرى لتركيا,"UNHQ Government and governance, public adminis..."
2,Open Details,Scientific and Technological Research Council ...,Conseil de la recherche scientifique et techno...,Consejo de Investigación Científica y Tecnológ...,NaN,NaN,المجلس التركي للبحوث العلمية والتكنولوجية,IMO Names of non-UN international organizations
3,Open Details,KGK Kurdistan Workers' Party PKK Workers' Part...,Parti des travailleurs du Kurdistan PKK,PTK Partido de los Trabajadores del Kurdistán ...,Конгра-Гел Рабочая партия Курдистана Курдская ...,库尔德斯坦工人党 劳工党 库工党 库尔德工人党,حزب العمال الكردي (الكردستاني),UNHQ Peacekeeping and peacebuildingSafety and ...


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Unnamed: 0                          4 non-null      object
 1   English  (Click to sort Ascending)  4 non-null      object
 2   French  (Click to sort Ascending)   4 non-null      object
 3   Spanish  (Click to sort Ascending)  4 non-null      object
 4   Russian  (Click to sort Ascending)  3 non-null      object
 5   Chinese  (Click to sort Ascending)  3 non-null      object
 6   Arabic  (Click to sort Ascending)   4 non-null      object
 7   Unnamed: 7                          4 non-null      object
dtypes: object(8)
memory usage: 388.0+ bytes


In [31]:
# Example usage
TEXT_TO_SEARCH = " the President of Türkiye"
data, markdown_table, table_html, df, data_df = search_term_and_extract_data(TEXT_TO_SEARCH)
print("\n\nDATA")
print(data)
print("\n\nMARKDOWN TABLE")
print(markdown_table)
print("\n\nTABLE HTML")
print(table_html)
print("\n\nDATAFRAME")
df

Update Default Settings button clicked.
Display in Spanish button clicked.
Display in Russian button clicked.
Display in Chinese button clicked.
Display in Arabic button clicked.
Failed to retrieve the table HTML content: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A2146CB5+28821]
	(No symbol) [0x00007FF7A20B3840]
	(No symbol) [0x00007FF7A1F5578A]
	(No symbol) [0x00007FF7A1FA91BE]
	(No symbol) [0x00007FF7A1FA94AC]
	(No symbol) [0x00007FF7A1FF2647]
	(No symbol) [0x00007FF7A1FCF33F]
	(No symbol) [0x00007FF7A1FEF412]
	(No symbol) [0x00007FF7A1FCF0A3]
	(No symbol) [0x00007FF7A1F9A778]
	(No symbol) [0x00007FF7A1F9B8E1]
	GetHandleVerifier [0x00007FF7A247FCAD+3408013]
	GetHandleVerifier [0x00007FF7A249741F+3504127]
	GetHandleVerifier [0x00007FF7A248B5FD+3455453]
	GetHandleVerifier [0x00007FF7A220BDBB+835995]
	(No symbol) [0x00007FF7A20BEB5F]
	(No symbol) [0x00007FF7A20BA814]
	(No symbol) [0x00007FF7A20BA9AD]
	(No symbol) [0x00007FF7A20AA199]
	BaseThreadInitThunk [0x00007FFDC5C97374+20

ValueError: not enough values to unpack (expected 5, got 0)

In [7]:
!pip install -U duckduckgo_search

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.9 MB 2.2 MB/s eta 0:00:02
   -- ------------------------------------- 0.2/2.9 MB 2.1 MB/s eta 0:00:02
   --- ------------------------------------ 0.3/2.9 MB 2.0 MB/s eta 0:00:02
   ----- ---------------------------------- 0.4/2.9 MB 1.9 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.9 MB 2.0 MB/s eta 0:00:02
   -------- ------------------------------- 0.6/2.9 MB 2.2 MB/s eta 0:00:02
   ---------- ----------------------------- 0.7/2.9 MB 2.4 MB/s eta 0:00:01
   ------------- -------------------------- 1.0/2.9 MB 2.5 MB/s eta 0:00:01
   --------------- ------------------------ 1.1/2.9 MB 2.7 MB/s eta 0:00:01
   --------------- ------------------------ 1.1/2.9 MB 2.5 MB/s eta 0:00:01
   ---------------- ----------------------- 1.2/2.9 MB 2.5 MB/s eta 0:00:01
   ------------------- -------------------- 1.4/2.9 MB 2.5 MB/s eta 0:00:01
   ----------------


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
def chat(self, keywords: str, model: str = "llama-3.1-70b", timeout: int = 30) -> str:
    """Initiates a chat session with DuckDuckGo AI.

    Args:
        keywords (str): The initial message or question to send to the AI.
        model (str): The model to use: "gpt-4o-mini", "claude-3-haiku", "llama-3.1-70b", "mixtral-8x7b".
            Defaults to "gpt-4o-mini".
        timeout (int): Timeout value for the HTTP client. Defaults to 30.

    Returns:
        str: The response from the AI.
    """

In [27]:
from duckduckgo_search import DDGS
results = DDGS().chat("summarize Daniel Defoe's The Consolidator", model='llama-3.1-70b')
results

'The Consolidator is a satirical novel written by Daniel Defoe, published in 1705. The story is set in a fantastical world where the protagonist, a traveler, journeys to the moon and discovers a utopian society. The novel is a commentary on the politics and society of Defoe\'s time, particularly in England.\n\nThe traveler meets various characters on the moon, including a group of people who have created a perfect society, free from the problems of Earth. They have a system of government that is fair and just, and they live in harmony with each other.\n\nHowever, the traveler soon discovers that this society is not as perfect as it seems. The inhabitants of the moon are obsessed with the idea of "consolidation," or the idea of bringing all things under one central authority. They believe that this is the key to achieving true happiness and perfection.\n\nThe novel is a satire of the idea of consolidation, which was a popular concept in Defoe\'s time. Defoe argues that the idea of conso